# Test the ENC-DEC model

In [1]:
import numpy as np
import dataloader
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import models
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import os
import time
from torch_geometric.nn import GATv2Conv, global_mean_pool
# reload library
import importlib
import cv2
#import utils as ut
import pandas as pd
import DataDLC
from torch_geometric.data import Data, DataLoader
import tqdm


# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [ ]:
importlib.reload(dataloader)
importlib.reload(DataDLC)
importlib.reload(models)

## Load dataset

**Obs:** This is just a test to see if the model is working.

In [ ]:
importlib.reload(dataloader)
importlib.reload(DataDLC)

In [ ]:

dataloader.reload_module()

In [5]:
# Load dataset .pt files
import pickle as pkl


In [ ]:
# deactivate warnings
if True:
    import warnings
    warnings.filterwarnings("ignore")

    data_loader = dataloader.DLCDataLoader(r'c:\Users\jalvarez\Documents\Data\LargeDataset', load_dataset=False, window_size=11, stride=1, build_graph = True)


In [7]:
data_loader.save_dataset()

**Load**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load the data
dataset = torch.load(r'c:\Users\jalvarez\Documents\Data\dataset_w1.pkl', map_location=device)

In [188]:
from torch_geometric.data import DataListLoader

In [182]:
seq_dataset = dataloader.SequenceDataset(dataset, sequence_length=5)


In [ ]:
data_loader = DataListLoader(seq_dataset, batch_size=10, shuffle=True)

In [191]:
a =next(iter(data_loader))


In [ ]:
len(a)

In [166]:
a = next(iter(dataloader))

In [ ]:
a

In [ ]:

# Create a new DataLoader for sequences
sequence_dataloader = DataLoader(sequence_dataset, batch_size=32, shuffle=True)

In [149]:
item = next(iter(sequence_dataloader))

In [ ]:
item

In [ ]:
dataset[0].behaviour_names

In [ ]:
len(dataset[0].behaviour_names)

In [14]:
if False:
    for i in range(len(dataset)):
        dataset[i].y = torch.tensor([dataset[i].behaviour[1]], dtype=torch.long)
        dataset[i].frame_mask = (dataset[i].frame_mask == 2).float()

In [ ]:
dataset[0].behaviour

**Augmentation**

In [16]:
import augmentation

In [ ]:
importlib.reload(augmentation)

In [ ]:
augmentation.merge_symetric_behaviours_version2?

In [17]:
idx_sniffR = 0
idx_sniffV = 13

In [ ]:
len(dataset)

In [19]:
# Suffle the dataset
np.random.seed(0)
np.random.shuffle(dataset)

In [20]:
# Split train and test
train_size = int(0.8 * len(dataset))

train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]

In [37]:
# See balance of classes
labels = [d.behaviour[idx_sniffR] for d in train_dataset]

In [ ]:
# Class balance
plt.hist(labels)
plt.show()

In [ ]:
print('Number of samples per class:')
print('Active:', sum([l == 1 for l in labels]))
print('Inactive:', sum([l == 0 for l in labels]))

In [31]:
augmentation.merge_symetric_behaviours_version2(indx_behaviour1=idx_sniffR, indx_behaviour2=idx_sniffV, dataset=train_dataset)
augmentation.rotate_samples(train_dataset, idx_sniffR)

In [ ]:
labels = [d.behaviour[idx_sniffR] for d in train_dataset]

# Class balance
plt.hist(labels)
plt.show()

In [ ]:
print('Number of samples per class:')
print('Active:', sum([l == 1 for l in labels]))
print('Inactive:', sum([l == 0 for l in labels]))

In [ ]:
# Compute weights for the loss function
weights = [1/sum([l == 0 for l in labels]), 1/sum([l == 1 for l in labels])]

weights

In [ ]:
# Compute weights for the loss function
import sklearn.utils as skutils

weights_ = skutils.class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=np.array(labels))
weights_

In [64]:
train_dataset = augmentation.downsample_majority_class(train_dataset, idx_sniffR)

In [ ]:
labels = [d.behaviour[idx_sniffR] for d in train_dataset]

# Class balance
plt.hist(labels)
plt.show()

In [ ]:
print('Number of samples per class:')
print('Active:', sum([l == 1 for l in labels]).item())
print('Inactive:', sum([l == 0 for l in labels]).item())

In [ ]:
labels = [d.behaviour[idx_sniffR] for d in test_dataset]

# Class balance
plt.hist(labels)
plt.show()


In [ ]:
print('Number of samples per class:')
print('Active:', sum([l == 1 for l in labels]).item())
print('Inactive:', sum([l == 0 for l in labels]).item())

In [26]:
# Keep only the idx_sniffR behaviour
for i in range(len(dataset)):
    dataset[i].behaviour = dataset[i].behaviour[idx_sniffR]

In [9]:
# Discard Nan's in the dataset
idx_to_keep = []
for i in range(len(dataset)):
    if dataset[i].behaviour != 0 and dataset[i].behaviour != 1:
        continue
    idx_to_keep.append(i)
    
        

In [10]:
dataset = [dataset[i] for i in idx_to_keep]

In [51]:
import sklearn.utils as skutils


In [ ]:
len(labels)

In [50]:
y = np.array(labels)

In [ ]:
y.sum()

In [ ]:
np.unique(y)

In [55]:
weights_ = skutils.class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)

In [ ]:
weights_

In [ ]:
# Class balance 
behaviour = [d.behaviour.item() for d in dataset]
behaviour = np.array(behaviour)
behaviour = np.unique(behaviour, return_counts=True)

plt.bar(behaviour[0], behaviour[1])
plt.xlabel('Behaviour')
plt.ylabel('Count')
plt.title('Class balance')
plt.show()

print('There are {} samples in the dataset'.format(len(dataset)))
print('There are {} samples of behaviour 0'.format(behaviour[1][0]))
print('There are {} samples of behaviour 1'.format(behaviour[1][1]))


**only an specific behaviour**

In [90]:
if False:
    for i in range(len(dataset)):
        dataset[i].behaviour = dataset[i].behaviour[1]


In [ ]:
dataset_cloned[0].behaviour

In [ ]:
len(dataset_cloned)

## Class Balance

In [ ]:
# Class Balance
behaviour = [d.behaviour[2].item() for d in dataset_cloned]
behaviour = np.array(behaviour)
behaviour = np.unique(behaviour, return_counts=True)
#Plot
plt.bar(behaviour[0][1:], behaviour[1][1:])
plt.xlabel('Behaviour')
plt.ylabel('Count')
plt.title('Class Balance')
plt.show()

print('There are {} instances of the behaviour'.format(behaviour[1][2]))

In [96]:
# Rotate the dataset
augmentation.rotate_samples(dataset_cloned, idx_sniffR)

In [ ]:
len(dataset_cloned)

In [ ]:
# Class Balance
behaviour = [d.behaviour[2].item() for d in dataset_cloned]
behaviour = np.array(behaviour)
behaviour = np.unique(behaviour, return_counts=True)
#Plot
plt.bar(behaviour[0][1:], behaviour[1][1:])
plt.xlabel('Behaviour')
plt.ylabel('Count')
plt.title('Class Balance')
plt.show()

print('There are {} instances of the behaviour'.format(behaviour[1][2]))

In [ ]:
ratio = behaviour[1][1] / behaviour[1][2]
print('The ratio between the two classes is {}'.format(ratio))

### Train-Test split

**Shuffle**

In [106]:
# Suffle the dataset
np.random.seed(0)
np.random.shuffle(dataset_cloned)

In [106]:
# Get only the first behaviour
for i in range(len(dataset)):
    dataset[i].behaviour = dataset[i].behaviour[0]

In [ ]:
# Split train and test
train_size = int(0.8 * len(dataset))

train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]

print('The train dataset has %d samples' % len(train_dataset))
print('The test dataset has %d samples' % len(test_dataset))

In [ ]:
dataset[-1].behaviour

### DataLoaders

In [ ]:
batch_size = 32

# Create the dataloaders for train, validation and test
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [28]:
for data in train_loader:
    a = data
    break

In [ ]:
# Plot first batch 
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()
G.add_nodes_from(range(a.num_nodes))
G.add_edges_from(a.edge_index.T.tolist())
G.nodes
#nx.draw(G, with_labels=True)



In [ ]:
G.edges

In [ ]:
a.edge_index

In [ ]:
# Plot first batch

nx.draw(G, with_labels=True)

---

# Create Graphs

This will take a while, and i dont even know if it is going to work. jejeje

In [ ]:
importlib.reload(models)

In [ ]:
len(dataset_cloned)

In [100]:
for data in train_loader:
    a = data
    break

In [ ]:
a.behaviour.shape

In [ ]:
a.frame_mask.unique()

### Graph Classifier

In [103]:
graphencoder = models.GATEncoder(nout = 64, nhid=32, attention_heads = 2, n_in = 4, n_layers=4, dropout=0.2)
class_head = models.ClassificationHead(n_latent=64, nhid = 32, nout = 2)

In [104]:
lat = graphencoder(a.x, a.edge_index)

In [ ]:
lat.shape

In [106]:
embbed = concatenate_per_graph(lat, a.batch, a.frame_mask)

In [ ]:
embbed.shape

In [ ]:
out = class_head(embbed)

In [ ]:
out.shape

In [ ]:
dataset[0]

In [111]:
def concatenate_per_graph(embbed, batch, frame_mask):
        ''' Concatenate the embeddings per graph '''
        out = []
        for i in range(batch.max()+1):
            out.append(embbed[batch==i][frame_mask[batch==i] == frame_mask[batch==i].median()].flatten())
        return torch.stack(out)

In [112]:
model = models.GraphClassifier(graphencoder, class_head)

In [113]:
out = model(a)

In [ ]:
out.shape

In [ ]:
print('The model has %d trainable parameters' % sum(p.numel() for p in model.parameters() if p.requires_grad))

In [116]:
import tqdm

#### Trainning Loop

In [117]:
# Trainning loop
num_epochs = 10
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter(log_dir='runs/TESTTTT')  # TensorBoard writer
actual_epoch = 0

In [ ]:
start_time = time.time()  # Time the training
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    correct_class_0 = 0
    correct_class_1 = 0
    total_class_0 = 0
    total_class_1 = 0
    total = 0
    i = 0

    for data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(data)
        labels = data.behaviour

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted = outputs.argmax(dim=1)
        correct_class_0 += (predicted[labels == 0] == labels[labels == 0]).sum().item()
        correct_class_1 += (predicted[labels == 1] == labels[labels == 1]).sum().item()
        total_class_0 += (labels == 0).sum().item()
        total_class_1 += (labels == 1).sum().item()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Log training loss and accuracy at each step
        if i % 100 == 0:  # Log every 100 iterations, adjust as needed
            writer.add_scalar('Loss/Train', loss.item(), (actual_epoch + epoch) * len(train_loader) + i)
            writer.add_scalar('Accuracy/Train', correct / total, (actual_epoch + epoch) * len(train_loader) + i)
        i += 1

    train_accuracy = correct / total
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + actual_epoch + 1}, Training Loss: {avg_train_loss}, Training Accuracy: {train_accuracy}")

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    correct_class_0 = 0
    correct_class_1 = 0
    total_class_0 = 0
    total_class_1 = 0
    total = 0
    with torch.no_grad():
        for val_data in tqdm.tqdm(test_loader):
            val_outputs = model(val_data)
            val_labels = val_data.behaviour
            val_loss += criterion(val_outputs, val_labels).item()
            val_predicted = val_outputs.argmax(dim=1)
            correct_class_0 += (val_predicted[val_labels == 0] == val_labels[val_labels == 0]).sum().item()
            correct_class_1 += (val_predicted[val_labels == 1] == val_labels[val_labels == 1]).sum().item()
            total_class_0 += (val_labels == 0).sum().item()
            total_class_1 += (val_labels == 1).sum().item()
            correct += (val_predicted == val_labels).sum().item()
            total += val_labels.size(0)

    val_accuracy = correct / total
    avg_val_loss = val_loss / len(test_loader)

    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}")

    # Log validation metrics
    writer.add_scalar('Loss/Validation', avg_val_loss, actual_epoch + epoch)
    writer.add_scalar('Accuracy/Validation', val_accuracy, actual_epoch + epoch)
    writer.add_scalar('Accuracy/Avarage_inactive_class_Validation', correct_class_0 / total_class_0, actual_epoch + epoch)
    writer.add_scalar('Accuracy/Avarage_active_class_Validation', correct_class_1 / total_class_1, actual_epoch + epoch)
    writer.add_scalar('Accuracy/Average_per_class_Validation', ((correct_class_0 / total_class_0) + (correct_class_1 / total_class_1)) / 2, actual_epoch + epoch)

    # Step the scheduler
    #scheduler.step()

    # Log learning rate
    current_lr = optimizer.param_groups[0]['lr']
    writer.add_scalar('Learning Rate', current_lr, actual_epoch + epoch)
    print(f"Learning Rate after epoch {epoch + 1}: {current_lr}")

    # Save checkpoint after each 5 epochs
    if (epoch + 1) % 10 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch + actual_epoch + 1}.pth')
        save_checkpoint(model, optimizer, epoch + 1, avg_train_loss, checkpoint_path)

# Save the final model
if num_epochs % 5 != 0:
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch + actual_epoch + 1}.pth')
    save_checkpoint(model, optimizer, epoch + 1, avg_train_loss, checkpoint_path)


---

## Load trained model


In [13]:
def save_checkpoint(model, optimizer, epoch, loss, path):
    # Save the model, optimizer state, epoch, and loss
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, path)
    print(f"Checkpoint saved at {path}")

In [14]:
# Load the model
def load_checkpoint(model, optimizer, path, device):
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded from {path}, at epoch {epoch}")
    return model, optimizer, epoch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load the data
dataset = torch.load(r'c:\Users\jalvarez\Documents\Data\LargeDataset\entire_dataset.pkl', map_location=device)

# Suffle the dataset
np.random.seed(0)
np.random.shuffle(dataset)



In [ ]:
len(dataset)

In [ ]:
dataset[0].behaviour_names


In [ ]:
dataset[0].behaviour

In [15]:
# get only the behaviour of interest
idx_to_keep = []
for i in range(len(dataset)):
    # If NaN, discard the sample
    if dataset[i].behaviour[3] != 0 and dataset[i].behaviour[3] != 1:
        continue
    else:
        dataset[i].behaviour = dataset[i].behaviour[3]
        idx_to_keep.append(i)

In [16]:
test_dataset = [dataset[i] for i in idx_to_keep]

In [ ]:
len(test_dataset)

In [18]:
# Delete the dataset
del dataset

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
importlib.reload(models)
# Define the model
graphencoder = models.GATEncoder(nout = 8, nhid=16, attention_hidden=2, n_in=4, dropout=0.5)
class_head = models.ClassificationHead(n_latent=576, nhid = 32, nout = 2)

model = models.GraphClassifier(graphencoder, class_head)

model.to(device)


In [21]:
checkpoint = torch.load(r'c:\Users\jalvarez\Documents\Data\Checkpoints\Dominance\checkpoint_epoch_160_without_weights', map_location=device)

In [ ]:
checkpoint['model_state_dict'].keys()

In [23]:
# Try on the entire dataset

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [24]:
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)


In [ ]:
# Load the model
checkpoint_path = r'c:\Users\jalvarez\Documents\Data\Checkpoints\Dominance\checkpoint_epoch_400.pth'
model, optimizer, start_epoch = load_checkpoint(model, optimizer, checkpoint_path, device)

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for val_data in tqdm.tqdm(test_loader):
        val_outputs = model(val_data)
        val_labels = val_data.behaviour
        val_predicted = val_outputs.argmax(dim=1)
        y_true.extend(val_labels.tolist())
        y_pred.extend(val_predicted.tolist())

In [ ]:
val_data.behaviour[1]

In [ ]:
print('Ther are %d samples' % len(y_true))
print('There are %d cases of not following' % y_true.count(0))
print('There are %d cases of following' % y_true.count(1))

In [ ]:
# accuracy
correct = sum([1 for i in range(len(y_true)) if y_true[i] == y_pred[i]])
accuracy = correct / len(y_true)
print(f"Accuracy: {accuracy}")

In [30]:
# Accuracy per class

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
# Percentage plot of the confusion matrix
cm = cm / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=['No Dominance', 'Dominance'], yticklabels=['No Dominance', 'Dominance'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Load the model
checkpoint_path = r'c:\Users\jalvarez\Documents\Data\Checkpoints\Grooming\checkpoint_epoch_190.pth'
model, optimizer, start_epoch = load_checkpoint(model, optimizer, checkpoint_path, device)

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for val_data in tqdm.tqdm(test_loader):
        val_outputs = model(val_data)
        val_labels = val_data.behaviour
        val_predicted = val_outputs.argmax(dim=1)
        y_true.extend(val_labels.tolist())
        y_pred.extend(val_predicted.tolist())

In [ ]:
print('Ther are %d samples' % len(y_true))
print('There are %d cases of not following' % y_true.count(0))
print('There are %d cases of following' % y_true.count(1))

# accuracy
correct = sum([1 for i in range(len(y_true)) if y_true[i] == y_pred[i]])
accuracy = correct / len(y_true)
print(f"Accuracy: {accuracy}")

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
# Percentage plot of the confusion matrix
cm = cm / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=['Not following', 'Following'], yticklabels=['Not Following', 'Following'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [122]:
import augmentation

In [ ]:
# Load the data
dataset = torch.load(r'c:\Users\jalvarez\Documents\Data\Dataset_DMDmaleMDX5CVmalefem\dataset_large.pkl', map_location=device)

# Select the behaviour to classify (Dominance in this case)
indx_behaviour1 = 3
indx_behaviour2 = 8

# Suffle the dataset
np.random.seed(0)
np.random.shuffle(dataset)


In [73]:
# get only the behaviour of interest
idx_to_keep = []
for i in range(len(dataset)):
    # If NaN, discard the sample
    if dataset[i].behaviour[2] != 0 and dataset[i].behaviour[2] != 1:
        continue
    else:
        idx_to_keep.append(i)

In [74]:
dataset = [dataset[i] for i in idx_to_keep]

In [25]:
# Suffle the dataset
np.random.seed(0)
np.random.shuffle(dataset)

In [26]:

# Split train and test
train_size = int(0.8 * len(dataset))

train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]


In [ ]:
print(len(dataset))
print(train_size)

In [ ]:


print('Merging the behaviours')
augmentation.merge_symetric_behaviours(indx_behaviour1, indx_behaviour2, train_dataset)
print('Generating rotation augmentation')
# Rotate the dataset
augmentation.rotate_samples(train_dataset, indx_behaviour1)
print('Downsampling the inactive behaviours')
train_dataset = augmentation.downsample_inactive(train_dataset, indx_behaviour1)

for i in range(len(train_dataset)):
    train_dataset[i].behaviour = train_dataset[i].behaviour[indx_behaviour1]
for i in range(len(test_dataset)):
    test_dataset[i].behaviour = test_dataset[i].behaviour[indx_behaviour1]
print('Done selecting the behaviour')


In [78]:
# Discard NaNs samples

In [ ]:
print('The train dataset has %d samples' % len(train_dataset))
print('The test dataset has %d samples' % len(test_dataset))

In [ ]:
# Class balance
behaviour = [d.behaviour[2] for d in train_dataset]
behaviour = np.array(behaviour)
behaviour = np.unique(behaviour, return_counts=True)
# Plot
plt.bar(behaviour[0], behaviour[1])
plt.xlabel('Behaviour')
plt.ylabel('Count')
plt.title('Class Balance')
plt.show()


In [ ]:
# weights
weights = skutils.class_weight.compute_class_weight('balanced', np.unique(behaviour[0]), behaviour[0])

In [ ]:
# class balance test
behaviour = [d.behaviour[2] for d in test_dataset]
behaviour = np.array(behaviour)
behaviour = np.unique(behaviour, return_counts=True)
# Plot
plt.bar(behaviour[0], behaviour[1])
plt.xlabel('Behaviour')
plt.ylabel('Count')
plt.title('Class Balance')
plt.show()


---

---

---

---

---

---

#### eDIT NAMES

In [23]:
path = r'c:\Users\jalvarez\Documents\Data\DEEPLABCUT_needs_this_folder_I_dont_like_dlc'

# replace ' ' by '_'
for filename in os.listdir(path):
    os.rename(os.path.join(path, filename), os.path.join(path, filename.replace(' ', '_')))

---

In [48]:
import pandas as pd

In [54]:
data_dlc = DataDLC.DataDLC(r'c:\Users\jalvarez\Documents\Data\DataLoadaerTESTTSTST\DMD_mal_Test_1DLC_dlcrnetms5_More_BodyPartsJul9shuffle1_740000_el_filtered.h5')

data_dlc.drop_tail_bodyparts()

coords = data_dlc.coords.to_numpy()

In [61]:
coords.shape
# Reshape coords as (n_frames, n_individuals, n_bodyparts, 3)
coords = coords.reshape((coords.shape[0], data_dlc.n_individuals, data_dlc.n_body_parts, 3))